### 데이터 불러오기

In [249]:
import pandas as pd
data = pd.read_csv(r'C:\Users\HwijunKwon\Downloads\RecipeData (1).csv')
recipe = data[['recipe_title', 'recipe_step', 'recipe_ingredients']] # 3가지 칼럼만 일단 사용


### 데이터 전처리

In [250]:
def combined_filter(data):
    def clean_ingredients(ingredient):
        return ingredient.replace('\\ufeff', '').replace('\\u200b', '')
    
    data['recipe_ingredients'] = clean_ingredients(data['recipe_ingredients']) #

    def not_empty_steps(data):
        return data['recipe_step'].strip() != '[]' # recipe_step의 결측치 제거
    def not_empty_ingredients(row):
        return data['recipe_ingredients'].strip() != '{}' # recipe_ingredients의 결측치 제거

    return not_empty_steps(data) & not_empty_ingredients(data)

recipe = recipe[recipe.apply(combined_filter, axis=1)]

### 레시피 STEP의 길이

In [246]:
import pandas as pd

bins = [0, 50, 10, 150, 20, 25, 30]
labels = [f'{i}자 이상' for i in bins[:-1]]

recipe['title_len'] = recipe['recipe_title'].astype(str).apply(len)

In [205]:
import pandas as pd
# 타이틀 길이 분포
bins = [0, 5, 10, 15, 20, 25, 30]
labels = [f'{i}자 이상' for i in bins[:-1]]

recipe['title_len'] = recipe['recipe_title'].astype(str).apply(len)
recipe['title_category'] = pd.cut(recipe['title_len'], bins = bins, labels = labels)

length_distribution = recipe['title_category'].value_counts().sort_index()
print(length_distribution)

0자 이상      7344
5자 이상     29371
10자 이상    30651
15자 이상    32758
20자 이상    27521
25자 이상    18446
Name: title_category, dtype: int64


In [208]:
# 레시피 길이 분포
bins = [0, 200, 400, 600, 800, 1000, 2000, 2500, 3000]
labels = [f'{i}자 이상' for i in bins[:-1]]

recipe['step_len'] = recipe['recipe_step'].astype(str).apply(len)
recipe['recipe_category'] = pd.cut(recipe['step_len'], bins = bins, labels = labels)

length_distribution = recipe['recipe_category'].value_counts().sort_index()
print(length_distribution)


0자 이상       32613
200자 이상     64095
400자 이상     39675
600자 이상     18092
800자 이상      7652
1000자 이상     6434
2000자 이상      212
2500자 이상       46
Name: recipe_category, dtype: int64


### 식재료 꺼내서 분류하기

In [251]:
from tqdm import tqdm
import ast
import re
import json

# 식재료 넣을 칼럼 생성하기 : 식재료, 양, 단위
for i in range(1, 25):
    recipe.loc[:, f'ingredient{i}'] = None
    recipe.loc[:, f'quantity{i}'] = None
    recipe.loc[:, f'unit{i}'] = None


In [261]:
non_matching_items = {}

for idx, row in tqdm(recipe.iterrows(), total = recipe.shape[0]):
    ingredient_count = 1
    if idx < 2000:
        ingredients_dict = ast.literal_eval(row["recipe_ingredients"]) # SyntaxError: unterminated string literal (detected at line 1)
        for items in ingredients_dict.values():
            for item in items :
                match = re.match(r'([가-힣]+(\([가-힣]+\))?)([\d.+/~-]*)([가-힣a-zA-Z]+|약간|조금)?', item)
                if match:
                    ingredient, _, quantity, unit = match.groups()
                    
                    recipe.loc[idx, f'ingredient{ingredient_count}'] = ingredient
                    recipe.loc[idx, f'quantity{ingredient_count}'] = quantity
                    recipe.loc[idx, f'unit{ingredient_count}'] = unit

                    ingredient_count += 1
                else:
                    non_matching_items[idx] = item

for idx, item in non_matching_items.items():
    print(f'Row {idx}: {item}')

  0%|          | 0/156701 [00:00<?, ?it/s]

100%|██████████| 156701/156701 [00:17<00:00, 9127.74it/s] 

Row 258: (소주or맛술)1큰술
Row 367: 18배멸치육수2T
Row 598: 
Row 837: 
Row 892: 
Row 1007: (볶을때)들기름약간
Row 1186: 
Row 1289: 
Row 1390: (머리와내장손질한)국물용멸치1줌
Row 1460: 
Row 1464: 
Row 1801: 


### 단위의 종류 찾기

In [211]:
units = [recipe[f'unit{i}'] for i in range(1, 25)]

In [217]:
combined_units = []
for unit_tuple in zip(*units):
    for unit in unit_tuple:
        if unit:  # 빈 값이 아닌 경우에만 추가
            combined_units.append(unit)

# 중복 제거
unique_units = list(set(combined_units))

from collections import Counter

unit_counts = Counter(combined_units)
for unit, count in unit_counts.items():
    print(f"{unit}: {count}번")

개: 1103번
큰술: 1609번
g: 421번
컵: 110번
줌: 86번
공기: 11번
줄: 16번
t: 153번
T: 1211번
쪽: 29번
통: 6번
팩: 33번
대: 75번
수저: 70번
작은수저: 2번
밥숟가락: 24번
봉지: 33번
모: 7번
그람: 7번
스푼: 692번
Ts: 77번
주먹: 20번
티스푼: 51번
대접: 5번
숟가락: 192번
캔: 14번
톨: 17번
접시: 2번
개정도: 8번
g정도: 3번
송이: 15번
알: 26번
작은술: 165번
뿌리: 19번
TS: 30번
조각: 7번
꼬집: 62번
수저씩: 2번
ml: 28번
g가량: 1번
테이블스푼: 4번
줄기: 9번
장: 118번
그램: 9번
웅큼: 3번
cc: 3번
종이컵: 11번
마리: 11번
방울: 3번
숟갈: 88번
봉: 17번
바구니: 1번
or부추적당량: 1번
움큼: 5번
스픈: 3번
소주잔: 8번
천원어치: 1번
덩이: 1번
조금: 3번
큰술반: 6번
근: 2번
ts: 20번
약간: 3번
사발: 1번
소주컵: 7번
단: 9번
번: 1번
개씩: 2번
술: 5번
스푼정도: 1번
cm: 9번
번톡톡: 3번
or물: 1번
큰술듬뿍: 1번
T듬뿍: 1번
통약간더: 1번
토막: 6번
tea스푼: 2번
센티: 1번
or연두: 1번
큰솔: 1번
묶음: 3번
개반: 2번
밥숟갈: 9번
cm정도: 1번
과: 2번
뼘: 3번
줌정도: 2번
알정도: 1번
장분량정도: 1번
국자: 3번
미리: 1번
포기: 10번
등분: 1번
와: 1번
분의: 5번
개또는반대: 1번
넉넉히: 1번
크술: 1번
잔: 1번
바퀴: 8번
or들기름: 1번
설탕: 1번
T정도: 2번
스푼반: 1번
타래: 1번
T스푼: 1번
g간장: 1번
머그컵: 1번
or물엿: 1번
대접정도: 1번
봉or: 1번
티스푼김치염도에따라조절: 1번
컵갈은깨: 1번
개작으면: 1번
없으면안넣어도무방: 1번
두꼬집: 1번
작은숟가락: 1번
소고기: 1번
청청양고추각: 1번
개작은것은: 1번
커피스푼: 1번
소주잔정도: 1번
g한봉지: 1번
or매

### 식재료 종류 찾기



In [263]:
ingreds = [recipe[f'ingredient{i}'] for i in range(1, 25)]

In [267]:
combined_ingred = []
for ingred_tuple in zip(*ingreds):
    for ingred in ingred_tuple:
        if ingred:  # 빈 값이 아닌 경우에만 추가
            combined_ingred.append(ingred)

# 중복 제거
unique_ingred = list(set(combined_ingred))

from collections import Counter

ingred_counts = Counter(combined_ingred)
for ingred, count in ingred_counts.items():
    print(f"{ingred}: {count}번")

애호박: 123번
다진마늘: 1012번
소금: 321번
참기름: 779번
식용유또는들기름: 1번
건새우: 43번
물: 368번
마늘종: 13번
설탕: 567번
간장: 789번
식용유: 475번
물엿: 166번
통깨: 405번
부추: 11번
달걀: 13번
밥: 2번
굴소스: 335번
베이컨: 31번
양파: 669번
미니새송이버섯: 7번
깻잎: 9번
대파: 303번
국간장: 76번
들깨가루: 61번
깨: 127번
꽈리고추: 94번
햄: 18번
진간장: 240번
마늘: 248번
올리고당: 614번
감자(중간사이즈): 3번
스팸: 14번
비엔나소세지: 41번
견과류약간: 2번
식용유약간: 75번
케찹: 42번
파프리카빨강이: 1번
파프리카노랑이: 1번
느타리버섯: 68번
올리브유: 65번
새우: 12번
마요네즈: 159번
버터: 17번
후추약간: 131번
소금약간: 84번
파슬리가루약간: 2번
우엉: 7번
당근: 203번
오일: 26번
미림: 64번
가지: 95번
청양고추: 237번
홍고추: 137번
굵은소금: 35번
고춧가루: 279번
메추리알: 2번
고추장: 284번
케첩: 27번
두부: 17번
다진소고기: 5번
맛술: 347번
다진파: 64번
후추톡톡톡: 9번
들기름: 304번
숙주: 17번
후추톡톡: 40번
참기름조금: 27번
다진쪽파: 5번
매실청: 72번
감자: 121번
맛간장(진간장): 1번
카놀라유: 39번
통깨조금: 51번
미니돈까스: 1번
단단한두부: 1번
고추장(평평하게깎아서계량): 1번
브로콜리: 24번
다시마물: 1번
액젓: 20번
쪽파: 37번
고추가루: 116번
깨소금약간: 72번
채썬햄: 1번
후추조금: 20번
깨조금: 25번
소고기: 7번
통깨툭툭: 1번
진미채: 99번
통깨적당량: 31번
새송이버섯: 58번
오일약간: 2번
참치: 13번
소금조금: 19번
시금치: 7번
양배추: 24번
통깨팍팍: 6번
참치캔: 9번
대파송송송: 2번
식초: 20번
오이: 23번
소고기다짐육: 5번
생강술: 10번
배즙: 2번
후춧가루적당량: 8번
전분가

In [268]:
ingred_counts.most_common

<bound method Counter.most_common of Counter({'다진마늘': 1012, '간장': 789, '참기름': 779, '양파': 669, '올리고당': 614, '설탕': 567, '식용유': 475, '통깨': 405, '물': 368, '맛술': 347, '굴소스': 335, '소금': 321, '들기름': 304, '대파': 303, '고추장': 284, '고춧가루': 279, '마늘': 248, '진간장': 240, '청양고추': 237, '통깨약간': 214, '당근': 203, '물엿': 166, '마요네즈': 159, '어묵': 159, '홍고추': 137, '후추약간': 131, '깨': 127, '애호박': 123, '감자': 121, '고추가루': 116, '깨소금': 110, '진미채': 99, '후추': 97, '가지': 95, '꽈리고추': 94, '새우젓': 91, '잔멸치': 85, '소금약간': 84, '사각어묵': 83, '멸치': 77, '국간장': 76, '식용유약간': 75, '매실청': 72, '깨소금약간': 72, '느타리버섯': 68, '올리브유': 65, '미림': 64, '다진파': 64, '청주': 64, '매실액': 64, '참기름약간': 62, '들깨가루': 61, '깨약간': 59, '새송이버섯': 58, '파': 57, '간마늘': 55, '통깨조금': 51, '후춧가루약간': 51, '꿀': 51, '당근약간': 49, '파프리카': 44, '건새우': 43, '마늘쫑': 43, '케찹': 42, '비엔나소세지': 41, '무': 41, '양조간장': 41, '후추톡톡': 40, '카놀라유': 39, '대파약간': 39, '표고버섯': 38, '당근조금': 38, '쪽파': 37, '미역줄기': 37, '포도씨유': 36, '기름': 36, '굵은소금': 35, '참깨': 34, '베이컨': 31, '통깨적당량': 31, '통마늘': 31, '콩나물': 30, '식용유적당량'

In [253]:
[recipe[f'ingredient{i}'] for i in range(1, 25)]

[0          애호박
 1          건새우
 2           부추
 3          베이컨
 4           깻잎
           ... 
 168819    None
 168820    None
 168821    None
 168822    None
 168824    None
 Name: ingredient1, Length: 156701, dtype: object,
 0         다진마늘
 1            물
 2           달걀
 3           양파
 4           대파
           ... 
 168819    None
 168820    None
 168821    None
 168822    None
 168824    None
 Name: ingredient2, Length: 156701, dtype: object,
 0              소금
 1             마늘종
 2             식용유
 3         미니새송이버섯
 4            다진마늘
            ...   
 168819       None
 168820       None
 168821       None
 168822       None
 168824       None
 Name: ingredient3, Length: 156701, dtype: object,
 0          참기름
 1           소금
 2            밥
 3         다진마늘
 4          국간장
           ... 
 168819    None
 168820    None
 168821    None
 168822    None
 168824    None
 Name: ingredient4, Length: 156701, dtype: object,
 0         식용유또는들기름
 1               설탕
 2              굴소스

In [218]:
from embedding import convert_fraction_to_float, convert_unit_to_number

ingredient_columns = recipe.filter(like='ingredient').drop(columns=['recipe_ingredients']) # 'ingredient'가 포함된 모든 열 선택
all_ingredients = [item for sublist in ingredient_columns.values for item in sublist if pd.notna(item)] # 각 행의 ingredient 값을 하나의 리스트로 결합

In [224]:
unique_ingred = list(set(all_ingredients))

In [225]:
unique_ingred

['참깨톡톡',
 '물(소주컵)',
 '맛간장',
 '볶음용멸치',
 '황설탕',
 '진미오징어채',
 '양배추약간',
 '구운소금',
 '다진마늘약간',
 '슬라이드아몬드',
 '비엔나소세지',
 '오징어',
 '굵은소금',
 '미니파프리카',
 '꽃소금',
 '식용류',
 '절이는소금',
 '실파약간',
 '불고기용소고기',
 '고추잎',
 '케찹',
 '양조간장',
 '고추기름',
 '갈은깨',
 '양파작',
 '다진청양고추',
 '납작오뎅',
 '대파송송',
 '깨소금(참기름)',
 '깨소금약간',
 '진미채오징어',
 '감자',
 '숙주나물',
 '후춧가루소량',
 '술',
 '브로콜리',
 '요리유',
 '홍청양고추',
 '부추나쪽파조금',
 '치즈가루(생략가능)',
 '통들깨약간',
 '맛술',
 '쪽파(대파)',
 '다시마우린물',
 '북어채또는황태채',
 '파약간',
 '파프리카각각',
 '김치',
 '묵은김치',
 '물종이컵',
 '통마늘원하는만큼',
 '마늘종(마늘속대)',
 '양파작은거',
 '올리고당',
 '대파손가락',
 '샐러리',
 '홍고추약간',
 '케첩',
 '쌀뜨물',
 '들깨',
 '공심채',
 '버터',
 '새송이버섯',
 '양파채',
 '파프리카색깔별로',
 '가지작은거',
 '쪽파약간',
 '느타리버섯적당량',
 '채썬햄',
 '소금톡톡',
 '밀가루소량',
 '맛소금약간',
 '팽이버섯',
 '시판스테이크(햄)',
 '깐마늘',
 '건곤드레나물',
 '무중',
 '묵은갓김치',
 '미니돈까스',
 '설탕(또는물엿)',
 '청양고추(생략가능)',
 '오이',
 '다시마',
 '물약간',
 '염장미역줄기',
 '참깨',
 '깨살짝',
 '후추(취향에따라)',
 '통깨또는다진땅콩적당량',
 '삼색파프리카',
 '고구마줄기',
 '마늘쫑약',
 '호박',
 '재료',
 '지리멸치적당량',
 '청주나미림',
 '고춧가루',
 '빅비엔나소세지',
 '식용유조금',
 '멸치(지리멸)',
 '깨적당량',
 '식용유(두르기)',
 '